In [ ]:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

In [1]:
import warnings
from dataclasses import dataclass
from typing import Any, Dict, List, Literal, Optional, Tuple, Union
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    GenerationConfig,
    PreTrainedModel,
)
from chronos import ChronosConfig, ChronosTokenizer, MeanScaleUniformBins, ChronosModel, ChronosPipeline

In [2]:
def left_pad_and_stack_1D(tensors: List[torch.Tensor]):
    max_len = max(len(c) for c in tensors)
    padded = []
    for c in tensors:
        assert isinstance(c, torch.Tensor)
        assert c.ndim == 1
        padding = torch.full(
            size=(max_len - len(c),), fill_value=torch.nan, device=c.device
        )
        padded.append(torch.concat((padding, c), dim=-1))
    return torch.stack(padded)


In [3]:
# from chronos import ChronosPipeline
import torch
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cpu",  # use "cpu" for CPU inference and "mps" for Apple Silicon
    torch_dtype=torch.bfloat16,
)

model=pipeline.model

C:\Users\poehlmann\PycharmProjects\BTW25\.venv\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset


# Load data
df = pd.read_csv("https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv")

context = torch.tensor(df["#Passengers"])


config = AutoConfig.from_pretrained("amazon/chronos-t5-small",
    device_map="cuda",  # use "cpu" for CPU inference and "mps" for Apple Silicon
    torch_dtype=torch.bfloat16)

chronos_config = ChronosConfig(**config.chronos_config)
tokenizer=chronos_config.create_tokenizer()

context_tensor = pipeline._prepare_and_validate_context(context)
token_ids, attention_mask, scale = tokenizer.context_input_transform(context_tensor)
scale


C:\Users\poehlmann\PycharmProjects\BTW25\.venv\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor([280.2986])

In [5]:
class UnivariateTimeSeriesDataset(Dataset):
    def __init__(self, series, sequence_length, prediction_length):
        """
        Args:
            series (Tensor): A tensor containing the time series data.
            sequence_length (int): The number of time steps to use as input.
            prediction_length (int): The number of time steps to predict.
        """
        self.series = series
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length

    def __len__(self):
        return len(self.series) - self.sequence_length - self.prediction_length

    def __getitem__(self, idx):
        '''
        idx is the starting point, we slice the sequence length for input
        and slice the prediction length for target output
        '''
        x = self.series[idx:idx + self.sequence_length]
        y = self.series[idx + self.sequence_length:idx + self.sequence_length + self.prediction_length]
        return x, y


sequence_length = 10  # Length of the input sequence
prediction_length = 5  # Length of the output sequence you want to predict
# Convert data array to a torch tensor
time_series_tensor = torch.tensor(df["#Passengers"], dtype=torch.float32)  

dataset = UnivariateTimeSeriesDataset(time_series_tensor, sequence_length, prediction_length)


batch_size = 4
train_data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)


In [ ]:
import torch.optim as optim

# Enable trainable parameters
model.train()

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in tqdm(range(5)):
    for inputs, targets in train_data_loader:
        optimizer.zero_grad()
        token_ids, attention_mask, scale = tokenizer.context_input_transform(inputs)

        samples = model(
                token_ids,
                attention_mask,
                prediction_length,
                num_samples=30,
                temperature=1.0,
                top_k=50,
                top_p=1.0,
            )

        predictions = []
        remaining = prediction_length

        while remaining > 0:
            token_ids, attention_mask, scale = tokenizer.context_input_transform(inputs)

            prediction = tokenizer.output_transform(
                samples, scale
            )

            predictions.append(prediction.median(dim=1))
            remaining -= prediction.shape[-1]

            if remaining <= 0:
                break

        preds = predictions[0].values.median(dim=1).values
        actuals = targets.median(dim=1).values
        loss = criterion(preds, actuals)
        if not loss.requires_grad:
            loss = loss.clone().requires_grad_(True)
            
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}: Loss = {loss.item()}')


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1: Loss = 24.34471321105957
Epoch 2: Loss = 1366.2958984375
Epoch 3: Loss = 116.33440399169922
Epoch 4: Loss = 926.5825805664062
